In [1]:
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.utils.vis_utils import plot_model
from keras.applications.inception_resnet_v2 import decode_predictions
from keras.layers.normalization import BatchNormalization

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os

import tensorflow
from time import time

import pandas

import pickle as pk

from keras.wrappers.scikit_learn import KerasClassifier

seed = 7

Using TensorFlow backend.


In [2]:
dir(keras.applications.inception_resnet_v2)

['InceptionResNetV2',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'absolute_import',
 'decode_predictions',
 'division',
 'inception_resnet_v2',
 'keras_modules_injection',
 'preprocess_input',
 'print_function']

In [3]:
np.random.seed(seed)

In [4]:
data_dir = '/data/oxford102/train'
img_width, img_height = 256, 256
batch_size = 32
epochs = 100
nr_categories = 102

In [5]:
img_width, img_height = 256, 256
train_data_dir = "/data/oxford102/train/"
validation_data_dir = "/data/oxford102/train/"
nb_train_samples = 4604
nb_validation_samples = 1094 

In [6]:
input_processor = keras.applications.inception_resnet_v2.preprocess_input

In [7]:
train_val_datagen = ImageDataGenerator(preprocessing_function=input_processor, 
                                           validation_split=0.2)

train_val_datagen_aug = ImageDataGenerator(
        #rescale=1. / 255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        preprocessing_function=input_processor,
        validation_split=0.2)

In [8]:

    
train_generator = train_val_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(img_width, img_height),  # all images will be resized to 250x250
        batch_size=batch_size,
        subset="training",
        class_mode='categorical')

validation_generator = train_val_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(img_width, img_height),  # all images will be resized to 250x250
        subset="validation",
        batch_size=batch_size,
        class_mode='categorical')


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.


In [9]:
#from keras import regularizers

k.set_learning_phase(0)

network_name = "inception_resnet_v2"
img_width, img_height = (256, 256)
if network_name == "vgg16":
    base_model = keras.applications.vgg16.VGG16(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
elif network_name == "vgg19":
    base_model = keras.applications.vgg19.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
elif network_name == "inception_resnet_v2":
    base_model = keras.applications.inception_resnet_v2.InceptionResNetV2(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
else:
    raise Exception("check your network name")

for layer in base_model.layers[:]:
    layer.trainable = False

    #Adding custom Layers 
k.set_learning_phase(1)
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation="relu", 
          #kernel_regularizer=regularizers.l2(0.01),
         #       activity_regularizer=regularizers.l1(0.001)
         )(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x, training=True)
#x = Dense(102, activation="relu")(x)
predictions = Dense(nr_categories, activation="softmax")(x)

_model = Model(input = base_model.input, output = predictions)

KeyboardInterrupt: 

In [ ]:
_model.summary()

In [10]:
# _model.compile(loss = "categorical_crossentropy", optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0), metrics=["accuracy"])


In [20]:
#run_training(model_name, _model, train_generator, validation_generator, params, num_train_img, num_val_img):
    

model_name = "inception_resnet_v2"
num_train_img = 4604
num_val_img = 1094
params = {'network_name': 'vgg19',
         'image_aug': False,
         'optimizer': 'SGD',
          'optimizer_params': None, 
         'batch_size': 512,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.001, momentum=0.9), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

Epoch 1/100
8/8 [==============================] - 12s 2s/step - loss: 5.1542 - acc: 0.0078 - val_loss: 4.6099 - val_acc: 0.0625

Epoch 00001: val_acc improved from -inf to 0.06250, saving model to /data/oxford102/experiments/1543119978.482745/inception_resnet_v2_1543119978.482745.h5
Epoch 2/100
8/8 [==============================] - 6s 736ms/step - loss: 4.5553 - acc: 0.0625 - val_loss: 4.0345 - val_acc: 0.1094

Epoch 00002: val_acc improved from 0.06250 to 0.10938, saving model to /data/oxford102/experiments/1543119978.482745/inception_resnet_v2_1543119978.482745.h5
Epoch 3/100
8/8 [==============================] - 6s 740ms/step - loss: 4.1362 - acc: 0.1758 - val_loss: 3.7754 - val_acc: 0.1875

Epoch 00003: val_acc improved from 0.10938 to 0.18750, saving model to /data/oxford102/experiments/1543119978.482745/inception_resnet_v2_1543119978.482745.h5
Epoch 4/100
8/8 [==============================] - 6s 727ms/step - loss: 3.6819 - acc: 0.1719 - val_loss: 3.5967 - val_acc: 0.2188

Epo

In [ ]:
#run_training(model_name, _model, train_generator, validation_generator, params, num_train_img, num_val_img):
    

model_name = "inception_resnet_v2"
num_train_img = 4604
num_val_img = 1094
params = {'network_name': 'inception_resnet_v2',
         'image_aug': False,
         'optimizer': 'SGD',
          'optimizer_params': None, 
         'batch_size': 256,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.001, momentum=0.9), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

In [ ]:
print(1)

In [21]:
model_name = "vgg19"
num_train_img = 4604
num_val_img = 1094

params = {'network_name': 'vgg19',
         'image_aug': False,
         'optimizer': 'SGD',
          'optimizer_params': None, 
         'batch_size': 128,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.001, momentum=0.9), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

Epoch 1/100
35/35 [==============================] - 28s 814ms/step - loss: 2.4109 - acc: 0.4500 - val_loss: 2.6102 - val_acc: 0.3672

Epoch 00001: val_acc improved from -inf to 0.36719, saving model to /data/oxford102/experiments/1543120266.6855826/vgg19_1543120266.6855826.h5
Epoch 2/100
35/35 [==============================] - 24s 687ms/step - loss: 2.2636 - acc: 0.4732 - val_loss: 2.4299 - val_acc: 0.4727

Epoch 00002: val_acc improved from 0.36719 to 0.47266, saving model to /data/oxford102/experiments/1543120266.6855826/vgg19_1543120266.6855826.h5
Epoch 3/100
35/35 [==============================] - 24s 690ms/step - loss: 2.1774 - acc: 0.4982 - val_loss: 2.1768 - val_acc: 0.5078

Epoch 00003: val_acc improved from 0.47266 to 0.50781, saving model to /data/oxford102/experiments/1543120266.6855826/vgg19_1543120266.6855826.h5
Epoch 4/100
35/35 [==============================] - 25s 715ms/step - loss: 2.0867 - acc: 0.5214 - val_loss: 2.1810 - val_acc: 0.5312

Epoch 00004: val_acc impr

In [22]:
model_name = "vgg19"
num_train_img = 4604
num_val_img = 1094
params = {'network_name': 'vgg19',
         'image_aug': False,
         'optimizer': 'SGD',
          'optimizer_params': None, 
         'batch_size': 64,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.001, momentum=0.9), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

Epoch 1/100
71/71 [==============================] - 53s 741ms/step - loss: 1.0126 - acc: 0.8127 - val_loss: 1.6782 - val_acc: 0.6176

Epoch 00001: val_acc improved from -inf to 0.61765, saving model to /data/oxford102/experiments/1543120827.3960307/vgg19_1543120827.3960307.h5
Epoch 2/100
71/71 [==============================] - 49s 690ms/step - loss: 0.9870 - acc: 0.8160 - val_loss: 1.5968 - val_acc: 0.6232

Epoch 00002: val_acc improved from 0.61765 to 0.62316, saving model to /data/oxford102/experiments/1543120827.3960307/vgg19_1543120827.3960307.h5
Epoch 3/100
71/71 [==============================] - 49s 693ms/step - loss: 0.8490 - acc: 0.8562 - val_loss: 1.4980 - val_acc: 0.6801

Epoch 00003: val_acc improved from 0.62316 to 0.68015, saving model to /data/oxford102/experiments/1543120827.3960307/vgg19_1543120827.3960307.h5
Epoch 4/100
71/71 [==============================] - 49s 692ms/step - loss: 0.8399 - acc: 0.8631 - val_loss: 1.4776 - val_acc: 0.6746

Epoch 00004: val_acc did 

In [23]:
model_name = "vgg19"
num_train_img = 4604
num_val_img = 1094
params = {'network_name': 'vgg19',
         'image_aug': False,
         'optimizer': 'SGD',
          'optimizer_params': None, 
         'batch_size': 32,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.001, momentum=0.9), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

Epoch 1/100
143/143 [==============================] - 103s 719ms/step - loss: 0.1352 - acc: 0.9963 - val_loss: 0.9897 - val_acc: 0.7757

Epoch 00001: val_acc improved from -inf to 0.77574, saving model to /data/oxford102/experiments/1543121994.541226/vgg19_1543121994.541226.h5
Epoch 2/100
143/143 [==============================] - 98s 687ms/step - loss: 0.1215 - acc: 0.9945 - val_loss: 0.9852 - val_acc: 0.7840

Epoch 00002: val_acc improved from 0.77574 to 0.78401, saving model to /data/oxford102/experiments/1543121994.541226/vgg19_1543121994.541226.h5
Epoch 3/100
143/143 [==============================] - 99s 691ms/step - loss: 0.1035 - acc: 0.9961 - val_loss: 0.9391 - val_acc: 0.7785

Epoch 00003: val_acc did not improve from 0.78401
Epoch 4/100
143/143 [==============================] - 99s 691ms/step - loss: 0.0918 - acc: 0.9972 - val_loss: 0.9372 - val_acc: 0.7739

Epoch 00004: val_acc did not improve from 0.78401
Epoch 5/100
143/143 [==============================] - 99s 691ms/s

# FINETUNE TOP LAYERS

In [26]:
for i, layer in enumerate(base_model.layers[619:]):
    print(i, layer.name)
    layer.trainable = True

0 batch_normalization_165
1 activation_165
2 conv2d_166
3 batch_normalization_166
4 activation_166
5 conv2d_164
6 conv2d_167
7 batch_normalization_164
8 batch_normalization_167
9 activation_164
10 activation_167
11 block8_1_mixed
12 block8_1_conv
13 block8_1
14 block8_1_ac
15 conv2d_169
16 batch_normalization_169
17 activation_169
18 conv2d_170
19 batch_normalization_170
20 activation_170
21 conv2d_168
22 conv2d_171
23 batch_normalization_168
24 batch_normalization_171
25 activation_168
26 activation_171
27 block8_2_mixed
28 block8_2_conv
29 block8_2
30 block8_2_ac
31 conv2d_173
32 batch_normalization_173
33 activation_173
34 conv2d_174
35 batch_normalization_174
36 activation_174
37 conv2d_172
38 conv2d_175
39 batch_normalization_172
40 batch_normalization_175
41 activation_172
42 activation_175
43 block8_3_mixed
44 block8_3_conv
45 block8_3
46 block8_3_ac
47 conv2d_177
48 batch_normalization_177
49 activation_177
50 conv2d_178
51 batch_normalization_178
52 activation_178
53 conv2d_17

In [27]:
model_name = "vgg19"
num_train_img = 4604
num_val_img = 1094
params = {'network_name': 'vgg19',
         'image_aug': False,
         'optimizer': 'SGD',
          'optimizer_params': None, 
         'batch_size': 64,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.001, momentum=0.9), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

Epoch 1/100
71/71 [==============================] - 161s 2s/step - loss: 4.5933 - acc: 0.0247 - val_loss: 4.5870 - val_acc: 0.0294

Epoch 00001: val_acc improved from -inf to 0.02941, saving model to /data/oxford102/experiments/1543122990.683244/vgg19_1543122990.683244.h5
Epoch 2/100
 6/71 [=>............................] - ETA: 2:02 - loss: 4.5906 - acc: 0.0312{'network_name': 'vgg19', 'image_aug': False, 'optimizer': 'SGD', 'optimizer_params': None, 'batch_size': 64, 'epochs': 100, 'image_size': (256, 256), 'log_time': 1543122990.683244}


KeyboardInterrupt: 